In [35]:
import numpy as np


def HestonCharFunc(u, kappa, rho, gamma, tau, v0, vbar, r):
    i = 1j
    D1 = np.sqrt((kappa - i * rho * gamma * u)**2 + (u**2 + i * u) * gamma**2)
    g = kappa - i * rho * gamma * u - D1 / (kappa - i * rho * gamma * u + D1)
    
    expr1_1 = i * u *r * tau
    expr1_2 = v0 / gamma**2 * ((1 - np.exp(-D1 * tau)) / (1 - g * np.exp(-D1 * tau)))
    expr1_3 = kappa - i * rho * gamma * u - D1
    
    expr2_1 = kappa * vbar / gamma**2
    expr2_2 = tau * (kappa - i * rho * gamma * u - D1)
    expr2_3 = 2 * np.log((1 - g * np.exp(-D1 * tau)) / (1 - g))
    
    expr1 = np.exp(expr1_1 + expr1_2 * expr1_3)
    expr2 = np.exp(expr2_1 * (expr2_2 - expr2_3))
    
    return expr1 * expr2


def ChiPsi(a, b, c, d, k):
    chi_1 = 1 / (1 + ((k * np.pi) / (b - a))**2)
    chi_2 = np.cos(k * np.pi * (d - a) / (b - a)) * np.exp(d)
    chi_3 = np.cos(k * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi_4 = k * np.pi / (b - a) * np.sin(k * np.pi * (d - a) / (b - a)) * np.exp(d)
    chi_5 = k * np.pi / (b - a) * np.sin(k * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi_1 * (chi_2 - chi_3 + chi_4 - chi_5)
    
    psi = np.zeros(k.shape)
    psi[1:] = (b - a) / (k[1:] * np.pi) * (np.sin(k[1:] * np.pi * (d - a) / (b - a)) - np.sin(k[1:] * np.pi * (c - a) / (b - a)))
    psi[0] = d - c
        
    return chi, psi

def HestonCOSCoefficient(a, b, k, opt_type='call'):
    if opt_type == 'call':
        c = 0
        d = b
        chi, psi = ChiPsi(a, b, c, d, k)
        term = chi - psi
        
    elif opt_type == 'put':
        c = a
        d = 0
        chi, psi = ChiPsi(a, b, c, d, k)
        term = psi - chi
        
    return 2 / (b - a) * term

def HestonCOSValue(S0, K, r, T, kappa, rho, gamma, v0, vbar, N=5000, L=8, opt_type='call'):
    i = 1j
    k = np.arange(0, N, 1)
    a = -L * np.sqrt(T)
    b = -a
    u = k * np.pi / (b - a)
    x = np.log(S0 / K)
    
    chfs = HestonCharFunc(u, kappa, rho, gamma, T, v0, vbar, r)
    U = HestonCOSCoefficient(a, b, k, opt_type)
    
    series = chfs * U * np.exp(i * k * np.pi * (x - a) / (b - a))
    series[0] = 0.5 * series[0]
    
    return K * np.exp(-r * T) * np.real(np.sum(series))

In [127]:
from scipy.stats import ncx2


def CIR_sample(vt, dt, gamma, kappa, vbar):
    c_bar = gamma**2 / (4 * kappa) * (1 - np.exp(-kappa * dt))
    delta = 4 * kappa * vbar / gamma**2
    kappa_bar = (4 * kappa * np.exp(-kappa * dt)) / (gamma**2 * (1 - np.exp(-kappa * dt))) * vt
    
    return c_bar * ncx2.rvs(delta, kappa_bar)


def DupireLocalVol(r, s, dvdt, dvdk, d2vdk2):
    numerator = dvdt + r * s * dvdk
    denominator = 0.5 * s**2 * d2vdk2
    return numerator / denominator


def ExpectationFromBins(S, v, n_bins=20):
    Sv = np.hstack([S, v])
    Sv_sorted = Sv[Sv[:, 0].argsort()]
    bin_size = int(S.shape[0] / n_bins)
    expectation = np.zeros([np.size(S), 2])
    
    for i in range(1, n_bins+1):
        sample = Sv_sorted[(i-1)*bin_size:i*bin_size, 1]
        expectation[(i-1)*bin_size:i*bin_size, 0] = Sv_sorted[(i-1)*bin_size:i*bin_size, 0]
        expectation[(i-1)*bin_size:i*bin_size, 1] = np.mean(sample)
        
    return expectation[:, 1].reshape([np.size(S), 1])

In [141]:
# Market parameters
gamma = 0.95
kappa = 1.05
rho = -0.315
vbar = 0.0855
v0 = 0.0945
S0 = 1
x0 = np.log(S0)
r = 0
T2 = 5

# SLV parameters
p = 0.25  # moderate
gammaModel = (1 - p) * gamma
kappaModel = (1 + p) * kappa
rhoModel = (1 + p) * rho
vbarModel = (1 - p) * vbar
v02 = (1 + p) * v0

# Monte Carlo parameters
n_paths = 100
t0 = 0
n_steps = 500
dt = T2 / n_steps

# Compute derivatives
t_bump = 1e-4
k_bump = 1e-4

V = lambda T, K: HestonCOSValue(S0, K, r, T, kappa, rho, gamma, v0, vbar, opt_type='call')
dV_dt = lambda T, K: (V(T + t_bump, K) - V(T, K)) / t_bump
dV_dK = lambda T, K: (V(T, K + k_bump) - V(T, K)) / k_bump
d2V_dK2 = lambda T, K: (V(T, K + k_bump) - 2 * V(T, K) + V(T, K - k_bump)) / (k_bump**2)

# Initialise arrays
S = S0 + np.zeros([n_paths, 1])
x = x0 + np.zeros([n_paths, 1])
M = 1.0 + np.zeros([n_paths, 1])  # money market account
v = v02 + np.zeros([n_paths, 1])  # SLV
v_new = v
v_old = v

t = 0

for i in range(1, n_steps):
    if i == 1:
        t_real = t
        t = 1 / n_steps
        
    dvdt = dV_dt(t, S)
    dvdk = dV_dK(t, S)
    d2vdk2 = np.maximum(np.abs(d2V_dK2(t, S)), 1e-7)  # make sure it is positive
    
    sigma_LV = np.sqrt(DupireLocalVol(r, S, dvdt, dvdk, d2vdk2))
    
    if t != 0:
        EV = ExpectationFromBins(S, v_new)
        sigma_new = sigma_LV2 / np.sqrt(EV)
    
    else:
        sigma_new = sigma_LV / np.sqrt(v0)
        sigma_old = sigma_new
    
    Z_x = np.random.normal(0.0, 1.0, [n_paths, 1])
    v_new[:, 0] = CIR_sample(v[:, 0] , dt, gammaModel, kappaModel, vbarModel)
    
    x = x + r * dt - 0.5 * sigma_old**2 * v_old * dt + rhoModel / gammaModel * sigma_old *\
                (v_new - kappaModel * vbarModel * dt + v_old * (kappaModel * dt - 1)) +\
                    np.sqrt(1 - rhoModel**2) * np.sqrt(sigma_old**2 * v_old * dt) * Z_x
    
    M = M + r * M * dt
    S = np.exp(x)
    
    # Moment matching
    a2_SLV = (S0 - np.mean(S / M)) / np.mean(1 / M)
    S = np.maximum(S + a2_SLV, 1e-4)
    
    if i == 1:
        t = t_real
        
    sigma_old = sigma_new
    v_old = v_new
    t += dt

In [142]:
# Now compute option price
def PriceFromMCPaths(S, K, T, r, opt_type='call'):
    K = np.array(K)
    result = np.zeros(K.shape)
    
    if opt_type == 'call':
        for idx, k in enumerate(K):
            result[idx] = np.exp(-r * T) * np.mean(np.maximum(S - k, 0))
            
    elif opt_type == 'put':
        for idx, k in enumerate(K):
            result[idx] = np.exp(-r * T) * np.mean(np.maximum(k - S, 0))
    
    return np.array(result)

strikes = [0.7, 1.0, 1.5]

PriceFromMCPaths(S, strikes, T2, r)

array([0.67553178, 0.60953178, 0.51547903])